Import necessary libraries

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import re

Read in CSV File

In [2]:
df = pd.read_csv('D:/Documents/REU/time_cleaned_30days-001.csv', low_memory=False)
df

,id,device_id,timestamp,action,extras,android_version,is_tablet,dayOfWeek,month,year
0,1,1,2020-03-04 01:24:39,android.intent.action.BATTERY_CHANGED,technology: Li-ionicon-small: 17303522max_char...,10,False,Wednesday,March,2020
1,2,1,2020-03-04 01:24:39,android.net.nsd.STATE_CHANGED,nsd_state: 2,10,False,Wednesday,March,2020
2,3,1,2020-03-04 01:24:39,android.net.conn.CONNECTIVITY_CHANGE,"networkInfo: [type: WIFI[], state: CONNECTED/C...",10,False,Wednesday,March,2020
3,4,1,2020-03-04 01:24:39,android.net.wifi.STATE_CHANGE,"networkInfo: [type: WIFI[], state: CONNECTED/C...",10,False,Wednesday,March,2020
4,5,1,2020-03-04 01:24:39,android.net.wifi.WIFI_STATE_CHANGED,previous_wifi_state: 2wifi_state: 3,10,False,Wednesday,March,2020
...,...,...,...,...,...,...,...,...,...,...
15474876,12179430,55,2019-11-25 02:04:13,android.intent.action.BATTERY_CHANGED,technology: Li-ionicon-small: 17303535max_char...,10,False,Monday,November,2019
15474877,12179431,55,2019-11-25 02:04:15,android.net.wifi.RSSI_CHANGED,newRssi: -60,10,False,Monday,November,2019
15474878,12179432,55,2019-11-25 02:04:16,android.intent.action.SCREEN_ON,NaN,10,False,Monday,November,2019
15474879,12179433,55,2019-11-25 02:04:16,android.intent.action.DREAMING_STOPPED,NaN,10,False,Monday,November,2019


Remove unnecessary columns. We just want device_id, timestamp, and action

In [3]:
df = df.drop(columns=['id', 'extras', 'android_version', 'is_tablet', 'dayOfWeek', 'month', 'year'], axis=1)
df

,device_id,timestamp,action
0,1,2020-03-04 01:24:39,android.intent.action.BATTERY_CHANGED
1,1,2020-03-04 01:24:39,android.net.nsd.STATE_CHANGED
2,1,2020-03-04 01:24:39,android.net.conn.CONNECTIVITY_CHANGE
3,1,2020-03-04 01:24:39,android.net.wifi.STATE_CHANGE
4,1,2020-03-04 01:24:39,android.net.wifi.WIFI_STATE_CHANGED
...,...,...,...
15474876,55,2019-11-25 02:04:13,android.intent.action.BATTERY_CHANGED
15474877,55,2019-11-25 02:04:15,android.net.wifi.RSSI_CHANGED
15474878,55,2019-11-25 02:04:16,android.intent.action.SCREEN_ON
15474879,55,2019-11-25 02:04:16,android.intent.action.DREAMING_STOPPED


Swap action and timestamp since this is the order needed for SPMF

In [4]:
cols = ['device_id', 'action', 'timestamp']
df = df.reindex(columns = cols)

In [5]:
df

,device_id,action,timestamp
0,1,android.intent.action.BATTERY_CHANGED,2020-03-04 01:24:39
1,1,android.net.nsd.STATE_CHANGED,2020-03-04 01:24:39
2,1,android.net.conn.CONNECTIVITY_CHANGE,2020-03-04 01:24:39
3,1,android.net.wifi.STATE_CHANGE,2020-03-04 01:24:39
4,1,android.net.wifi.WIFI_STATE_CHANGED,2020-03-04 01:24:39
...,...,...,...
15474876,55,android.intent.action.BATTERY_CHANGED,2019-11-25 02:04:13
15474877,55,android.net.wifi.RSSI_CHANGED,2019-11-25 02:04:15
15474878,55,android.intent.action.SCREEN_ON,2019-11-25 02:04:16
15474879,55,android.intent.action.DREAMING_STOPPED,2019-11-25 02:04:16


Remove battery events

In [6]:
df = df.drop(df[df['action'].str.contains('BATTERY')].index)
df

,device_id,action,timestamp
1,1,android.net.nsd.STATE_CHANGED,2020-03-04 01:24:39
2,1,android.net.conn.CONNECTIVITY_CHANGE,2020-03-04 01:24:39
3,1,android.net.wifi.STATE_CHANGE,2020-03-04 01:24:39
4,1,android.net.wifi.WIFI_STATE_CHANGED,2020-03-04 01:24:39
5,1,android.net.wifi.p2p.STATE_CHANGED,2020-03-04 01:24:39
...,...,...,...
15474843,55,android.intent.action.SCREEN_OFF,2019-11-25 02:02:35
15474877,55,android.net.wifi.RSSI_CHANGED,2019-11-25 02:04:15
15474878,55,android.intent.action.SCREEN_ON,2019-11-25 02:04:16
15474879,55,android.intent.action.DREAMING_STOPPED,2019-11-25 02:04:16


I want to fix the dataframe index now that the rows with battery events have been removed.

In [7]:
df.reset_index(inplace=True)
df

,index,device_id,action,timestamp
0,1,1,android.net.nsd.STATE_CHANGED,2020-03-04 01:24:39
1,2,1,android.net.conn.CONNECTIVITY_CHANGE,2020-03-04 01:24:39
2,3,1,android.net.wifi.STATE_CHANGE,2020-03-04 01:24:39
3,4,1,android.net.wifi.WIFI_STATE_CHANGED,2020-03-04 01:24:39
4,5,1,android.net.wifi.p2p.STATE_CHANGED,2020-03-04 01:24:39
...,...,...,...,...
7887051,15474843,55,android.intent.action.SCREEN_OFF,2019-11-25 02:02:35
7887052,15474877,55,android.net.wifi.RSSI_CHANGED,2019-11-25 02:04:15
7887053,15474878,55,android.intent.action.SCREEN_ON,2019-11-25 02:04:16
7887054,15474879,55,android.intent.action.DREAMING_STOPPED,2019-11-25 02:04:16


The dataframe index is ok now but a new index column was added. Get rid of that.

In [8]:
df = df.drop(columns=['index'], axis=1)
df

,device_id,action,timestamp
0,1,android.net.nsd.STATE_CHANGED,2020-03-04 01:24:39
1,1,android.net.conn.CONNECTIVITY_CHANGE,2020-03-04 01:24:39
2,1,android.net.wifi.STATE_CHANGE,2020-03-04 01:24:39
3,1,android.net.wifi.WIFI_STATE_CHANGED,2020-03-04 01:24:39
4,1,android.net.wifi.p2p.STATE_CHANGED,2020-03-04 01:24:39
...,...,...,...
7887051,55,android.intent.action.SCREEN_OFF,2019-11-25 02:02:35
7887052,55,android.net.wifi.RSSI_CHANGED,2019-11-25 02:04:15
7887053,55,android.intent.action.SCREEN_ON,2019-11-25 02:04:16
7887054,55,android.intent.action.DREAMING_STOPPED,2019-11-25 02:04:16


Much better. Next let's work on the timestamp column. The software requires a single number, so we will convert it to epoch time.

In [9]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = (df['timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds()
df

,device_id,action,timestamp
0,1,android.net.nsd.STATE_CHANGED,1.583285e+09
1,1,android.net.conn.CONNECTIVITY_CHANGE,1.583285e+09
2,1,android.net.wifi.STATE_CHANGE,1.583285e+09
3,1,android.net.wifi.WIFI_STATE_CHANGED,1.583285e+09
4,1,android.net.wifi.p2p.STATE_CHANGED,1.583285e+09
...,...,...,...
7887051,55,android.intent.action.SCREEN_OFF,1.574647e+09
7887052,55,android.net.wifi.RSSI_CHANGED,1.574647e+09
7887053,55,android.intent.action.SCREEN_ON,1.574647e+09
7887054,55,android.intent.action.DREAMING_STOPPED,1.574647e+09


Next is to map each action to an integer so the software can process it. Make a dictionary.

In [10]:
actions = df['action'].unique()
len(actions)

79

In [11]:
vals = list(range(1, 80))
themap = dict(zip(actions, vals)) 

In [12]:
for k, v in themap.items():
    print('@ITEM={}={}'.format(v, k))

@ITEM=1=android.net.nsd.STATE_CHANGED
@ITEM=2=android.net.conn.CONNECTIVITY_CHANGE
@ITEM=3=android.net.wifi.STATE_CHANGE
@ITEM=4=android.net.wifi.WIFI_STATE_CHANGED
@ITEM=5=android.net.wifi.p2p.STATE_CHANGED
@ITEM=6=android.net.wifi.supplicant.STATE_CHANGE
@ITEM=7=android.media.ACTION_SCO_AUDIO_STATE_UPDATED
@ITEM=8=android.media.RINGER_MODE_CHANGED
@ITEM=9=android.media.SCO_AUDIO_STATE_CHANGED
@ITEM=10=android.net.wifi.RSSI_CHANGED
@ITEM=11=android.net.wifi.supplicant.CONNECTION_CHANGE
@ITEM=12=android.bluetooth.adapter.action.SCAN_MODE_CHANGED
@ITEM=13=android.bluetooth.adapter.action.STATE_CHANGED
@ITEM=14=android.net.wifi.SCAN_RESULTS
@ITEM=15=android.bluetooth.headset.profile.action.CONNECTION_STATE_CHANGED
@ITEM=16=android.bluetooth.a2dp.profile.action.CONNECTION_STATE_CHANGED
@ITEM=17=android.bluetooth.adapter.action.CONNECTION_STATE_CHANGED
@ITEM=18=android.intent.action.CONFIGURATION_CHANGED
@ITEM=19=android.speech.tts.TTS_QUEUE_PROCESSING_COMPLETED
@ITEM=20=android.intent.act

Map each action to the corresponding integer from the dictionary.

In [13]:
df = df.replace({"action":themap})
df

,device_id,action,timestamp
0,1,1,1.583285e+09
1,1,2,1.583285e+09
2,1,3,1.583285e+09
3,1,4,1.583285e+09
4,1,5,1.583285e+09
...,...,...,...
7887051,55,21,1.574647e+09
7887052,55,10,1.574647e+09
7887053,55,23,1.574647e+09
7887054,55,22,1.574647e+09


Before we split it, we need to remove duplicates.

In [14]:
df = df.drop_duplicates(subset = ['action', 'timestamp'], keep = 'first')
df

,device_id,action,timestamp
0,1,1,1.583285e+09
1,1,2,1.583285e+09
2,1,3,1.583285e+09
3,1,4,1.583285e+09
4,1,5,1.583285e+09
...,...,...,...
7886985,55,22,1.574648e+09
7886986,55,24,1.574648e+09
7886987,55,14,1.574648e+09
7886988,55,20,1.574648e+09


Now we need to sort it by device_id first, and then by timestamp.

In [15]:
df = df.sort_values(by=['device_id', 'timestamp'], ascending=[True, True])
df

,device_id,action,timestamp
6065,1,28,1.583284e+09
6066,1,22,1.583284e+09
6067,1,23,1.583284e+09
6068,1,14,1.583284e+09
6069,1,27,1.583284e+09
...,...,...,...
7871362,81,10,1.574771e+09
7871363,81,10,1.574771e+09
7871364,81,10,1.574771e+09
7871365,81,10,1.574771e+09


Checking how many unique devices we have

In [17]:
dev_id = df['device_id'].unique()
len(dev_id)

58

I'm going to export to a csv before I start breaking it down by user.

In [84]:
df.to_csv(r'C:/Users/User/Desktop/timestamp/edited_30_days.csv', index = False)

In [27]:
testdf = pd.read_csv('C:/Users/User/Desktop/timestamp/edited_30_days.csv')
testdf

,device_id,action,timestamp
0,1,28,1.583284e+09
1,1,22,1.583284e+09
2,1,23,1.583284e+09
3,1,14,1.583284e+09
4,1,27,1.583284e+09
...,...,...,...
2349077,81,10,1.574771e+09
2349078,81,10,1.574771e+09
2349079,81,10,1.574771e+09
2349080,81,10,1.574771e+09


Group the actions that happen at the same timestamp into one row.

In [28]:
new_df = testdf.groupby(['device_id', 'timestamp'])['action'].apply(list).reset_index(name = 'action')

In [87]:
new_df

,device_id,timestamp,action
0,1,1.583284e+09,"[28, 22]"
1,1,1.583284e+09,[23]
2,1,1.583284e+09,[14]
3,1,1.583284e+09,[27]
4,1,1.583284e+09,"[20, 21]"
...,...,...,...
1848079,81,1.574771e+09,[10]
1848080,81,1.574771e+09,[10]
1848081,81,1.574771e+09,[10]
1848082,81,1.574771e+09,[10]


Swap the columns so the action comes first and then the timestamp

In [88]:
cols = ['device_id', 'action', 'timestamp']
new_df = new_df.reindex(columns = cols)
new_df

,device_id,action,timestamp
0,1,"[28, 22]",1.583284e+09
1,1,[23],1.583284e+09
2,1,[14],1.583284e+09
3,1,[27],1.583284e+09
4,1,"[20, 21]",1.583284e+09
...,...,...,...
1848079,81,[10],1.574771e+09
1848080,81,[10],1.574771e+09
1848081,81,[10],1.574771e+09
1848082,81,[10],1.574771e+09


In [89]:
new_df['action'] = new_df['action'].apply(lambda x: ' '.join(map(str, x)))
new_df['timestamp'] = new_df['timestamp'].astype(str).apply(lambda x: x.replace ('.0', ''))
new_df

,device_id,action,timestamp
0,1,28 22,1583283937
1,1,23,1583283938
2,1,14,1583283940
3,1,27,1583283942
4,1,20 21,1583283948
...,...,...,...
1848079,81,10,1574771271
1848080,81,10,1574771295
1848081,81,10,1574771328
1848082,81,10,1574771368


action | timestamp is the format that SPMF needs

In [90]:
new_df['combined'] = new_df['action'].astype(str) + '|' + new_df['timestamp'].astype(str)
new_df

,device_id,action,timestamp,combined
0,1,28 22,1583283937,28 22|1583283937
1,1,23,1583283938,23|1583283938
2,1,14,1583283940,14|1583283940
3,1,27,1583283942,27|1583283942
4,1,20 21,1583283948,20 21|1583283948
...,...,...,...,...
1848079,81,10,1574771271,10|1574771271
1848080,81,10,1574771295,10|1574771295
1848081,81,10,1574771328,10|1574771328
1848082,81,10,1574771368,10|1574771368


Now we can get rid of everything except the device id and the combined column

In [91]:
new_df = new_df.drop(columns=['action', 'timestamp'], axis=1)
new_df

,device_id,combined
0,1,28 22|1583283937
1,1,23|1583283938
2,1,14|1583283940
3,1,27|1583283942
4,1,20 21|1583283948
...,...,...
1848079,81,10|1574771271
1848080,81,10|1574771295
1848081,81,10|1574771328
1848082,81,10|1574771368


Let's export to a text file just so we have it saved at this point

In [92]:
new_df.to_csv(r'C:/Users/User/Desktop/timestamp/combined_cols_30_days.txt', index = False)

In [96]:
df2 = pd.read_csv('C:/Users/User/Desktop/timestamp/combined_cols_30_days.txt', low_memory=False)
df2

,device_id,combined
0,1,28 22|1583283937
1,1,23|1583283938
2,1,14|1583283940
3,1,27|1583283942
4,1,20 21|1583283948
...,...,...
1848079,81,10|1574771271
1848080,81,10|1574771295
1848081,81,10|1574771328
1848082,81,10|1574771368


This is exactly what I need. Now I can get each device's data into their own dataframes and files.

Go through each device number, making a dataframe for it

In [276]:
device81 = new_df.loc[new_df['device_id'] == 81]

Almost done. Just need to get rid of the device_id column

In [277]:
device81 = device81.drop(columns=['device_id'], axis=1)

Now we can save it to a .txt

In [278]:
device81.to_csv(r'C:/Users/User/Desktop/split_by_user_30_days/device81.txt', index = False, header = False)

There are only 58 devices, so checking which ones are missing

In [97]:
devices = new_df['device_id'].unique()
devices

array([ 1,  2,  4,  5,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21,
       22, 24, 25, 26, 28, 30, 33, 35, 36, 38, 39, 40, 41, 46, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 69,
       70, 71, 72, 74, 75, 79, 81], dtype=int64)